This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [15]:
import os

import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "voc-1398922860126677349004466dd8e4d6e8d10.19282078"

In [16]:
import openai
print(openai.__version__)

0.28.1


In [2]:
!pip install lancedb
!pip install ipywidgets
!pip install tantivy
!pip install markdown
!pip install bs4
!pip install inflect
!pip install pandas

  Using cached lancedb-0.13.0-cp38-abi3-win_amd64.whl (23.7 MB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
  Using cached pylance-0.17.0-cp39-abi3-win_amd64.whl (27.0 MB)
  Using cached retry-0.9.2-py2.py3-none-any.whl (8.0 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
  Using cached pyarrow-17.0.0-cp310-cp310-win_amd64.whl (25.1 MB)
  Using cached py-1.11.0-py2.py3-none-any.whl (98 kB)



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached tantivy-0.22.0-cp310-none-win_amd64.whl (2.6 MB)



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


...


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached inflect-7.4.0-py3-none-any.whl (34 kB)
  Using cached typeguard-4.3.0-py3-none-any.whl (35 kB)
  Using cached more_itertools-10.5.0-py3-none-any.whl (60 kB)



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import openai
print(openai.__version__)

0.28.1


In [18]:
import os
import json
import re
import pandas as pd
import numpy as np
import openai

from bs4 import BeautifulSoup
from markdown import markdown

from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import LanceDB

from lancedb.rerankers import LinearCombinationReranker
import inflect
import lancedb
from lancedb.embeddings import get_registry
from lancedb.pydantic import LanceModel, Vector

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, FloatSlider

### load keys and files

In [19]:
db_file = "home_match.json"


In [20]:
system_prompt = """
You are an expert real estate agent in Los Angeles in the USA.
"""

human_prompt= """
      Generate a minimum of 10 real estate listings using creativity and incorporating real properties as well.
      Ensure the listings are distributed across all the areas and regions within the Los Angeles City. 
      Each listing should be represented as a JSON array of dictionaries, following the structure provided below:

        {
        "location": "Downtown Los Angeles",
        "list_price": 1800000,
        "bedrooms": 3,
        "bathrooms": 2,
        "square_feet": 1800,
        "school_rating": 4.5,
        "description": "Luxurious penthouse with floor-to-ceiling windows, boasting stunning city views. Features a gourmet kitchen, spacious open layout, and private balcony. Building amenities include a rooftop pool, gym, and 24-hour concierge."
       }
    """

if os.path.isfile(path=db_file):
  f = open(db_file, "r")
  real_estate_listings = f.read()
  f.close()
else:
  chat = ChatOpenAI(temperature=1)
  messages = [
      SystemMessage(
          content=system_prompt
      ),
      HumanMessage(
          content=human_prompt
      ),
  ]
  aimessage = chat.invoke(messages)
  real_estate_listings = json.loads(aimessage.json())["content"]

md_text = markdown(real_estate_listings)
real_estate_listings_text = ''.join(BeautifulSoup(md_text).findAll(string=True))
real_estate_listings_json = real_estate_listings_text.replace('json\n', '')
print(real_estate_listings_json)
f = open(db_file, "w")
f.write(real_estate_listings_json)
f.close()

[
  {
    "location": "Downtown Los Angeles",
    "list_price": 1800000,
    "bedrooms": 3,
    "bathrooms": 2,
    "square_feet": 1800,
    "school_rating": 4.5,
    "description": "Luxurious penthouse with floor-to-ceiling windows, boasting stunning city views. Features a gourmet kitchen, spacious open layout, and private balcony. Building amenities include a rooftop pool, gym, and 24-hour concierge."
  },
  {
    "location": "Hollywood",
    "list_price": 2500000,
    "bedrooms": 4,
    "bathrooms": 4,
    "square_feet": 2400,
    "school_rating": 4.6,
    "description": "Modern villa nestled in the Hollywood Hills, featuring a home theater, infinity pool, and panoramic views of the city. This home is a true entertainer's dream, complete with high-end finishes and smart home technology."
  },
  {
    "location": "Santa Monica",
    "list_price": 3200000,
    "bedrooms": 5,
    "bathrooms": 4,
    "square_feet": 3000,
    "school_rating": 4.8,
    "description": "Stunning beachfront 

In [21]:
df = pd.read_json(db_file)

# Connect to LanceDB
db = lancedb.connect("real-estate-embeddings-db")

# Define the schema for the real estate listings
class RealEstateListings(LanceModel):
    location: str
    list_price: float
    bedrooms: int
    bathrooms: int
    square_feet: float
    school_rating: float
    description: str
    description_vector: Vector(1536)

# Check if the listings table exists
if 'listings' in db.table_names():
  table = db.open_table("listings")
else:
  table = db.create_table("listings", schema=RealEstateListings)
    
  # Prepare data for insertion
  data = []
  for _, row in df.iterrows():
    # Directly call OpenAI API to get embeddings for the description
    response = openai.Embedding.create(
            model="text-embedding-ada-002",  # Specify the model
            input=row["description"]
        )
        
    # Extract the embedding vector
    embedding_vector = response['data'][0]['embedding']
        
    # Append data for the table
    data.append({
        "location": row["location"],
        "list_price": row["list_price"],
        "bedrooms": row["bedrooms"],
        "bathrooms": row["bathrooms"],
        "square_feet": row["square_feet"],
        "school_rating": row["school_rating"],
        "description": row["description"],
        "description_vector": embedding_vector
    })
  table.add(data)
print(table.to_pandas().head())

               location  list_price  bedrooms  bathrooms  square_feet  \
0  Downtown Los Angeles   1800000.0         3          2       1800.0   
1             Hollywood   2500000.0         4          4       2400.0   
2          Santa Monica   3200000.0         5          4       3000.0   
3          Sherman Oaks   1450000.0         3          2       2000.0   
4           Silver Lake   1700000.0         4          3       2200.0   

   school_rating                                        description  \
0            4.5  Luxurious penthouse with floor-to-ceiling wind...   
1            4.6  Modern villa nestled in the Hollywood Hills, f...   
2            4.8  Stunning beachfront property with private acce...   
3            4.7  Charming ranch-style home in the heart of the ...   
4            4.4  Architectural gem with modern design and eco-f...   

                                  description_vector  
0  [0.0045068543, 0.0021039487, 0.008447941, -0.0...  
1  [-0.008148264, -0.000

In [ ]:
#### User Interface 

In [ ]:

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value='Max Price'), FloatSlider(min=1000000, max=5000000, step=10000, value=5000000)], layout=form_item_layout),
    Box([Label(value='Bed Rooms minimum'), FloatSlider(min=1, max=10, step=1)], layout=form_item_layout),
    Box([Label(value='Bath Rooms minimum'), FloatSlider(min=1, max=10, step=1)], layout=form_item_layout),
    Box([Label(value='School Ratings'), FloatSlider(min=1, max=5, step=1)], layout=form_item_layout),
    Box([Label(value='Square Footage'), FloatSlider(min=1000, max=5000, step=500)], layout=form_item_layout),
    Box([Label(value='Preferences'),
         Textarea(value="brownstone prewar historic")], layout=form_item_layout)
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))
form
